# Importing packages 


In [1]:
# Array math 
import numpy as np 

# Q Learning 

Q learning is an off-policy TD algorithm to approximate the best optimal policy. 

Definitions: 

*Q value* - the value given for an action $a$ taken in state $s$. To put it more mathematically, it maps the actions and states to the real number plane: 

$$Q(a \in \mathbb{A}, s \in \mathbb{S}) \rightarrow q \in  \mathbb{R}$$

In practise, the bigger the q value for an action is, the "better" it is for the agent to take. 

*Q table* - a matrix storing the Q values where each row is a state and each column is an action. 

*$\epsilon$-greedy policy* - the policy where, at every decision point, the agent takes a random action with the probability of 1 - $\epsilon$ and the action which has the biggest q value with the probability of $\epsilon$, where $\epsilon \in [0, 1]$.

*off-policy algorithm* - algorithms where the agent updates it's policy not using it's behavior policy. 

*behavior policy* - the policy used by the agent to make an action.

Full algorithm for Q learning: 

1. Hyperparameters: 

1.1 Define step size $\alpha \in (0, 1]$

1.2 Define $\epsilon \in [0, 1]$

1.3 Define the discount factor $\gamma \in (0, 1]$

1.4 Initialize the Q table where all the values are arbitraty except for terminal states $Q(terminal, *) = 0$

1.5 Define the number of episodes $N$. 

2. Iterate for 1 to $N$:

2.1 Pick a starting state s.

2.2 Iterate until the agent reaches a terminal state: 

2.2.1 From the given state, pick an action $A$ using epsilon-greedy policy 

2.2.2 Take action $A$, observe the transition state $s^{'}$ and the reward $r$

**2.2.3 Update the current Q value estimate:** 

$$ Q(S, A) \leftarrow Q(S, A) + \alpha \left(r + \gamma \max_{a} Q(s^{'}, a) - Q(S, A)\right)$$

2.2.4 Set $s$ $\leftarrow$ $s^{'}$

The 2.2.3 update is the main part of the Q algorithm. 

As we can see, in step 2.2.1, the behavior policy is the epslion-greedy policy. 

But when we update the current Q value, we do not use that behaviour policy - we update the current Q value from the future state Q value and not use the epsilon-greedy policy anywhere. 